# Cleaning and Preprocessing

Import

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

df_sales = pd.read_excel('/content/Pizza_Sale.xlsx')

print(df_sales.shape)
df_sales.head(10)

(48620, 12)


,pizza_id,order_id,pizza_name_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name
0,1,1,hawaiian_m,1,2015-01-01 00:00:00,11:38:36,13.25,13.25,M,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese",The Hawaiian Pizza
1,2,2,classic_dlx_m,1,2015-01-01 00:00:00,11:57:40,16.00,16.00,M,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,...",The Classic Deluxe Pizza
2,3,2,five_cheese_l,1,2015-01-01 00:00:00,11:57:40,18.50,18.50,L,Veggie,"Mozzarella Cheese, Provolone Cheese, Smoked Go...",The Five Cheese Pizza
3,4,2,ital_supr_l,1,2015-01-01 00:00:00,11:57:40,20.75,20.75,L,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni...",The Italian Supreme Pizza
4,5,2,mexicana_m,1,2015-01-01 00:00:00,11:57:40,16.00,16.00,M,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O...",The Mexicana Pizza
5,6,2,thai_ckn_l,1,2015-01-01 00:00:00,11:57:40,20.75,20.75,L,Chicken,"Chicken, Pineapple, Tomatoes, Red Peppers, Tha...",The Thai Chicken Pizza
6,7,3,ital_supr_m,1,2015-01-01 00:00:00,12:12:28,16.50,16.50,M,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni...",The Italian Supreme Pizza
7,8,3,prsc_argla_l,1,2015-01-01 00:00:00,12:12:28,20.75,20.75,L,Supreme,"Prosciutto di San Daniele, Arugula, Mozzarella...",The Prosciutto and Arugula Pizza
8,9,4,ital_supr_m,1,2015-01-01 00:00:00,12:16:31,16.50,16.50,M,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni...",The Italian Supreme Pizza
9,10,5,ital_supr_m,1,2015-01-01 00:00:00,12:21:30,16.50,16.50,M,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni...",The Italian Supreme Pizza


In [ ]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48620 entries, 0 to 48619
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   pizza_id           48620 non-null  int64  
 1   order_id           48620 non-null  int64  
 2   pizza_name_id      48604 non-null  object 
 3   quantity           48620 non-null  int64  
 4   order_date         48620 non-null  object 
 5   order_time         48620 non-null  object 
 6   unit_price         48620 non-null  float64
 7   total_price        48613 non-null  float64
 8   pizza_size         48620 non-null  object 
 9   pizza_category     48597 non-null  object 
 10  pizza_ingredients  48607 non-null  object 
 11  pizza_name         48613 non-null  object 
dtypes: float64(2), int64(3), object(7)
memory usage: 4.5+ MB


Handling Missing Values

In [ ]:
df_sales.isna().sum()

,0
pizza_id,0
order_id,0
pizza_name_id,16
quantity,0
order_date,0
order_time,0
unit_price,0
total_price,7
pizza_size,0
pizza_category,23


In [ ]:
# pizza_ingredients
df_sales.loc[df_sales['pizza_ingredients'].isnull(), 'pizza_ingredients'] = (
    df_sales.groupby(['pizza_category', 'pizza_name'])['pizza_ingredients']
    .transform(lambda x: x.ffill().bfill()))

# pizza_name
df_sales.loc[df_sales['pizza_name'].isnull(), 'pizza_name'] = (
    df_sales.groupby(['pizza_name_id'])['pizza_name']
    .transform(lambda x: x.ffill().bfill()))

# pizza_category
df_sales.loc[df_sales['pizza_category'].isnull(), 'pizza_category'] = (
    df_sales.groupby(['pizza_ingredients'])['pizza_category']
    .transform(lambda x: x.ffill().bfill()))

# pizza_name_id
df_sales.loc[df_sales['pizza_name_id'].isnull(), 'pizza_name_id'] = (
    df_sales.groupby(['pizza_size', 'pizza_name'])['pizza_name_id']
    .transform(lambda x: x.ffill().bfill()))

# total_price
df_sales.loc[df_sales['total_price'].isnull(), 'total_price'] = (
    df_sales['quantity'] * df_sales['unit_price'])

df_sales.isna().sum()

,0
pizza_id,0
order_id,0
pizza_name_id,0
quantity,0
order_date,0
order_time,0
unit_price,0
total_price,0
pizza_size,0
pizza_category,0


Checking for Duplicates

In [ ]:
df_sales.duplicated().sum()

np.int64(0)

# Feature Engineering

In [ ]:
df_sales['order_date'] = pd.to_datetime(df_sales['order_date'], format='mixed')
df_sales['order_time'] = pd.to_datetime(df_sales['order_time'], format='%H:%M:%S').dt.time

# --- Time-based features ---
df_sales['day_of_week'] = df_sales['order_date'].dt.day_name()
df_sales['day_of_week_num'] = df_sales['order_date'].dt.weekday  # Monday=0, Sunday=6
df_sales['day_of_year'] = df_sales['order_date'].dt.dayofyear

# Weekend -> Saturday(5) or Sunday(6)
df_sales['is_weekend'] = df_sales['day_of_week_num'].isin([5, 6]).astype(int)

df_sales['month'] = df_sales['order_date'].dt.month_name()
df_sales['month_num'] = df_sales['order_date'].dt.month

df_sales['week_of_year'] = ((df_sales['day_of_year'] - 1) // 7) + 1

# Start of the week (Monday)
df_sales['week_start_date'] = df_sales['order_date'] - pd.to_timedelta(df_sales['day_of_week_num'], unit='D')

# --- Hour & Time bucket (optional for EDA) ---
df_sales['order_hour'] = pd.to_datetime(df_sales['order_time'], format='%H:%M:%S', errors='coerce').dt.hour

df_sales['day_of_month'] = df_sales['order_date'].dt.day
df_sales['quarter'] = df_sales['order_date'].dt.quarter
df_sales['is_month_start'] = df_sales['order_date'].dt.is_month_start.astype(int)
df_sales['is_month_end'] = df_sales['order_date'].dt.is_month_end.astype(int)

def time_bucket(h):
    if pd.isna(h):
        return pd.NA
    if 5 <= h <= 11:
        return 'Morning'
    elif 12 <= h <= 16:
        return 'Afternoon'
    elif 17 <= h <= 21:
        return 'Evening'
    else:
        return 'Night'
df_sales['time_bucket'] = df_sales['order_hour'].apply(time_bucket)

# Numeric mapping for time buckets
time_bucket_mapping = {
    'Morning': 1,
    'Afternoon': 3,
    'Evening': 2,
    'Night': 0}

# Apply the mapping
df_sales['time_bucket_num'] = df_sales['time_bucket'].map(time_bucket_mapping)

df_sales.columns

Index(['pizza_id', 'order_id', 'pizza_name_id', 'quantity', 'order_date',
       'order_time', 'unit_price', 'total_price', 'pizza_size',
       'pizza_category', 'pizza_ingredients', 'pizza_name', 'day_of_week',
       'day_of_week_num', 'day_of_year', 'is_weekend', 'month', 'month_num',
       'week_of_year', 'week_start_date', 'order_hour', 'day_of_month',
       'quarter', 'is_month_start', 'is_month_end', 'time_bucket',
       'time_bucket_num'],
      dtype='object')

In [ ]:
df_sales.shape

(48620, 27)

In [ ]:
df_sales.to_csv('Prepped_Sales.csv', index=False)

# Prepping for Training

In [ ]:
import pandas as pd

df = pd.read_csv('/content/Prepped_Sales.csv')
print(df.columns)
print(df.shape)
df.head()

Index(['pizza_id', 'order_id', 'pizza_name_id', 'quantity', 'order_date',
       'order_time', 'unit_price', 'total_price', 'pizza_size',
       'pizza_category', 'pizza_ingredients', 'pizza_name', 'day_of_week',
       'day_of_week_num', 'day_of_year', 'is_weekend', 'month', 'month_num',
       'week_of_year', 'week_start_date', 'order_hour', 'day_of_month',
       'quarter', 'is_month_start', 'is_month_end', 'time_bucket',
       'time_bucket_num'],
      dtype='object')
(48620, 27)


,pizza_id,order_id,pizza_name_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name,day_of_week,day_of_week_num,day_of_year,is_weekend,month,month_num,week_of_year,week_start_date,order_hour,day_of_month,quarter,is_month_start,is_month_end,time_bucket,time_bucket_num
0,1,1,hawaiian_m,1,2015-01-01,11:38:36,13.25,13.25,M,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese",The Hawaiian Pizza,Thursday,3,1,0,January,1,1,2014-12-29,11,1,1,1,0,Morning,1
1,2,2,classic_dlx_m,1,2015-01-01,11:57:40,16.00,16.00,M,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,...",The Classic Deluxe Pizza,Thursday,3,1,0,January,1,1,2014-12-29,11,1,1,1,0,Morning,1
2,3,2,five_cheese_l,1,2015-01-01,11:57:40,18.50,18.50,L,Veggie,"Mozzarella Cheese, Provolone Cheese, Smoked Go...",The Five Cheese Pizza,Thursday,3,1,0,January,1,1,2014-12-29,11,1,1,1,0,Morning,1
3,4,2,ital_supr_l,1,2015-01-01,11:57:40,20.75,20.75,L,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni...",The Italian Supreme Pizza,Thursday,3,1,0,January,1,1,2014-12-29,11,1,1,1,0,Morning,1
4,5,2,mexicana_m,1,2015-01-01,11:57:40,16.00,16.00,M,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O...",The Mexicana Pizza,Thursday,3,1,0,January,1,1,2014-12-29,11,1,1,1,0,Morning,1


Few of the following columns can be dropped as it adds no value for sales forecasting:

pizza_id             -- Just like an index, adds no value for model training  
pizza_ingredients    -- Adds no value for sales forecasting model training  
pizza_name           -- We have this in pizza_name_id  
day_of_week & month  -- We have this in numeric values  
week_start_date      -- Adds no value for sales forecasting  
total_price          -- Adds no value for sales forecast  
order_time           -- Adds no value as we have alnertative column  
time_bucket          -- Adds no value as we have alnertative numeric column

In [ ]:
df['order_date'] = pd.to_datetime(df['order_date'])
df = df.sort_values(['pizza_name_id', 'order_date']).reset_index(drop=True)

df.drop(['pizza_id', 'order_time', 'total_price', 'pizza_ingredients', 'pizza_name', 'day_of_week',
       'month', 'week_start_date', 'time_bucket', 'order_id', 'day_of_year'], axis=1, inplace=True)

Encoding

In [ ]:
# Categorical mappings
# =========================
subcat_map = {'Chicken':1, 'Supreme':0, 'Veggie':0, 'Classic':0}
df['sub_category'] = df['pizza_category'].map(subcat_map)

size_map = {'S':1, 'M':2, 'L':3, 'XL':4, 'XXL':5}
df['pizza_size'] = df['pizza_size'].map(size_map)

category_map = {'Classic':1, 'Supreme':2, 'Veggie':3, 'Chicken':4}
df['pizza_category'] = df['pizza_category'].map(category_map)

In [ ]:
df.shape

(48620, 17)

In [ ]:
df.columns

Index(['pizza_name_id', 'quantity', 'order_date', 'unit_price', 'pizza_size',
       'pizza_category', 'day_of_week_num', 'is_weekend', 'month_num',
       'week_of_year', 'order_hour', 'day_of_month', 'quarter',
       'is_month_start', 'is_month_end', 'time_bucket_num', 'sub_category'],
      dtype='object')

In [ ]:
df.to_csv('train_data_without_scaling.csv', index=False)

Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler

columns_to_scale = ['unit_price', 'pizza_size',
       'pizza_category', 'day_of_week_num', 'month_num',
       'week_of_year', 'order_hour', 'day_of_month', 'quarter',
      'time_bucket_num']

scaler = MinMaxScaler()

df_scaled = df.copy()

scaler.fit(df_scaled[columns_to_scale])
df_scaled[columns_to_scale] = scaler.transform(df_scaled[columns_to_scale])

In [ ]:
df.head(10)

,pizza_name_id,quantity,order_date,unit_price,pizza_size,pizza_category,day_of_week_num,is_weekend,month_num,week_of_year,order_hour,day_of_month,quarter,is_month_start,is_month_end,time_bucket_num,sub_category
0,bbq_ckn_l,1,2015-01-01,20.75,3,4,3,0,1,1,13,1,1,1,0,3,1
1,bbq_ckn_l,1,2015-01-01,20.75,3,4,3,0,1,1,13,1,1,1,0,3,1
2,bbq_ckn_l,1,2015-01-01,20.75,3,4,3,0,1,1,15,1,1,1,0,3,1
3,bbq_ckn_l,1,2015-01-01,20.75,3,4,3,0,1,1,15,1,1,1,0,3,1
4,bbq_ckn_l,1,2015-01-01,20.75,3,4,3,0,1,1,18,1,1,1,0,2,1
5,bbq_ckn_l,1,2015-01-01,20.75,3,4,3,0,1,1,22,1,1,1,0,0,1
6,bbq_ckn_l,1,2015-01-02,20.75,3,4,4,0,1,1,15,2,1,0,0,3,1
7,bbq_ckn_l,1,2015-01-02,20.75,3,4,4,0,1,1,17,2,1,0,0,2,1
8,bbq_ckn_l,1,2015-01-02,20.75,3,4,4,0,1,1,19,2,1,0,0,2,1
9,bbq_ckn_l,1,2015-01-02,20.75,3,4,4,0,1,1,21,2,1,0,0,2,1


In [ ]:
df_scaled.head(10)

,pizza_name_id,quantity,order_date,unit_price,pizza_size,pizza_category,day_of_week_num,is_weekend,month_num,week_of_year,order_hour,day_of_month,quarter,is_month_start,is_month_end,time_bucket_num,sub_category
0,bbq_ckn_l,1,2015-01-01,0.419847,0.5,1.0,0.500000,0,0.0,0.0,0.285714,0.000000,0.0,1,0,1.000000,1
1,bbq_ckn_l,1,2015-01-01,0.419847,0.5,1.0,0.500000,0,0.0,0.0,0.285714,0.000000,0.0,1,0,1.000000,1
2,bbq_ckn_l,1,2015-01-01,0.419847,0.5,1.0,0.500000,0,0.0,0.0,0.428571,0.000000,0.0,1,0,1.000000,1
3,bbq_ckn_l,1,2015-01-01,0.419847,0.5,1.0,0.500000,0,0.0,0.0,0.428571,0.000000,0.0,1,0,1.000000,1
4,bbq_ckn_l,1,2015-01-01,0.419847,0.5,1.0,0.500000,0,0.0,0.0,0.642857,0.000000,0.0,1,0,0.666667,1
5,bbq_ckn_l,1,2015-01-01,0.419847,0.5,1.0,0.500000,0,0.0,0.0,0.928571,0.000000,0.0,1,0,0.000000,1
6,bbq_ckn_l,1,2015-01-02,0.419847,0.5,1.0,0.666667,0,0.0,0.0,0.428571,0.033333,0.0,0,0,1.000000,1
7,bbq_ckn_l,1,2015-01-02,0.419847,0.5,1.0,0.666667,0,0.0,0.0,0.571429,0.033333,0.0,0,0,0.666667,1
8,bbq_ckn_l,1,2015-01-02,0.419847,0.5,1.0,0.666667,0,0.0,0.0,0.714286,0.033333,0.0,0,0,0.666667,1
9,bbq_ckn_l,1,2015-01-02,0.419847,0.5,1.0,0.666667,0,0.0,0.0,0.857143,0.033333,0.0,0,0,0.666667,1


In [ ]:
df_scaled.to_csv('training_dataset.csv', index=False)

# Cleaning Ingredients Dataset

In [ ]:
import pandas as pd

ing_df = pd.read_excel('/content/Pizza_ingredients.xlsx')

print(ing_df.shape)
ing_df.head(10)

In [ ]:
ing_df.isna().sum()

In [ ]:
ing_df[ing_df['Items_Qty_In_Grams'].isnull()]

In [ ]:
ing_df[(ing_df['pizza_name_id'] == 'brie_carre_s') & (ing_df['pizza_ingredients'] == 'Caramelized Onions')]

In [ ]:
ing_df[(ing_df['pizza_ingredients'] == 'Caramelized Onions')]

In [ ]:
ing_df[ing_df['pizza_ingredients'] == 'Sliced Ham']

In [ ]:
ing_df.dropna(inplace=True)

In [ ]:
ing_df.isna().sum()

In [ ]:
ing_df.shape

In [ ]:
ing_df.to_csv('Prepped_Ingredients.csv', index = False)